In [1]:
import torch
import numpy as np
from torch import nn
from torch import optim
import os

import torch.nn.functional as F
from torchvision import datasets ,transforms, models
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
from torch.autograd import Variable



In [2]:
data_dir= 'data/train'

def load_split_train_test(datadir,valid_size=.2):
    # create a tranformation pipeline , all the images go 
    #under all the tranformations
    
    train_transforms=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),])
    test_transforms=transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),])
    
    # callable funtions for loadeing data from image forlders,
    #apllying transformatioon
    train_data=  datasets.ImageFolder(datadir, transform=train_transforms)
    test_data=  datasets.ImageFolder(datadir,transform=test_transforms)
    
    # spliting train and test indices
    num_train= len(train_data)
    indices=list(range(num_train))
    split=int(np.floor(valid_size*num_train))
    
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx=indices[split:],indices[:split]
    
    train_sampler= SubsetRandomSampler(train_idx)
    test_sampler= SubsetRandomSampler(test_idx)
    
    trainloader=torch.utils.data.DataLoader(train_data,sampler=train_sampler, 
                                            batch_size=128)
    testloader=torch.utils.data.DataLoader(test_data,sampler=train_sampler, 
                                            batch_size=128)
    
    return trainloader,testloader

trainloader,testloader=load_split_train_test(data_dir,.2)

print(trainloader.dataset.classes)

for batch_images, targets in trainloader:
    print(batch_images.shape)  # should be 32*3*224*224


    
    
    
    
    
    
    
    
    
    

['with_mask', 'without_mask']
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([128, 3, 224, 224])
torch.Size([77, 3, 224, 224])


In [61]:
# check if gpu is present or not 
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model= models.resnet50(pretrained=True)
print(model)



cuda
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride

)


In [62]:
os.environ['CUDA_LAUNCH_BLOCKING']='0'
torch.cuda.get_device_name(torch.cuda.current_device())
torch.cuda.set_device(0)

In [63]:
for param in model.parameters():
    param.requires_grad=False
# New layer for classification
model.fc=nn.Sequential(nn.Linear(2048,512),
                       nn.ReLU(),
                       nn.Dropout(0.2),
                       nn.Linear(512,10),
                       nn.LogSoftmax(dim=1))

criterion=nn.NLLLoss()
optimizer=optim.Adam(model.fc.parameters(),lr=0.003)

model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [64]:

epochs=20
steps=0

running_loss=0

print_every=5

train_losses,test_losses=[],[]

for epoch in range(epochs):
    for inputs,labels in trainloader:
        steps+=1
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        logps=model.forward(inputs)
        loss=criterion(logps,labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()
        
        if steps%print_every==0:
            test_loss=0
            accuracy=0
            for inputs,labels in testloader:
                inputs,labels=inputs.to(device),labels.to(device)
                logps=model.forward(inputs)
                batch_loss=criterion(logps,labels)
                test_loss += batch_loss.item()
                ps=torch.exp(logps)
                
                top_p,top_class=ps.topk(1,dim=1)
                equals= top_class==labels.view(*top_class.shape)
                accuracy+=torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss/len(trainloader))
            test_losses.append(test_loss/len(testloader))
            
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss=0
            model.train()
            
            
torch.save(model,'classifymodel.pth')
            

Epoch 1/20.. Train loss: 1.501.. Test loss: 0.129.. Test accuracy: 0.959
Epoch 1/20.. Train loss: 0.418.. Test loss: 0.179.. Test accuracy: 0.927
Epoch 1/20.. Train loss: 0.297.. Test loss: 0.059.. Test accuracy: 0.974
Epoch 2/20.. Train loss: 0.155.. Test loss: 0.051.. Test accuracy: 0.982
Epoch 2/20.. Train loss: 0.186.. Test loss: 0.064.. Test accuracy: 0.973
Epoch 2/20.. Train loss: 0.044.. Test loss: 0.122.. Test accuracy: 0.950
Epoch 2/20.. Train loss: 0.207.. Test loss: 0.160.. Test accuracy: 0.958
Epoch 3/20.. Train loss: 0.032.. Test loss: 0.125.. Test accuracy: 0.954
Epoch 3/20.. Train loss: 0.086.. Test loss: 0.022.. Test accuracy: 0.992
Epoch 3/20.. Train loss: 0.065.. Test loss: 0.030.. Test accuracy: 0.991
Epoch 4/20.. Train loss: 0.007.. Test loss: 0.047.. Test accuracy: 0.982
Epoch 4/20.. Train loss: 0.032.. Test loss: 0.015.. Test accuracy: 0.992
Epoch 4/20.. Train loss: 0.012.. Test loss: 0.023.. Test accuracy: 0.990
Epoch 4/20.. Train loss: 0.031.. Test loss: 0.040..

KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses,label='Traning Loss')
plt.plot(test_losses,label='Validation Loss')

plt.legend(frameon=False)
plt.show()
